<a href="https://colab.research.google.com/github/yaolihua081/COVID-19-Project/blob/main/USACovid_19_By_Race.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#
import seaborn as sns

import sys, platform, numpy as np, nltk, re, sklearn, pandas as pd, seaborn as sns, collections


import matplotlib.pyplot as plt
import hashlib
import matplotlib.style as style
style.use('fivethirtyeight')

import altair as alt
#https://data.cdc.gov/NCHS/Provisional-COVID-19-Death-Counts-by-Sex-Age-and-S/9bhg-hcku
#census population
#https://usafacts.org/data/topics/people-society/population-and-demographics/population-data/population/?utm_source=google&utm_medium=cpc&utm_campaign=ND-Race&gclid=CjwKCAiA-_L9BRBQEiwA-bm5fjq2Tta61ZG4veY5qG9W0YW3OjX19QrnxCH24i9rWYXK4hB_dF0YARoCCS0QAvD_BwE#chart-114185-1
import folium
import datetime

This data is for Geo Plot

In [44]:

#df = pd.read_excel('https://query.data.world/s/dyp652e6sdd6p5slgyuotz2egaorje')
#df.head()
from vega_datasets import data
source = data.population_engineers_hurricanes()
source
states = alt.topo_feature(data.us_10m.url, 'states')
states
# Stae name and their IDS, with their populations
source_sorted=source.sort_values(by='state')
source_sorted.head()


,state,id,population,engineers,hurricanes
0,Alabama,1,4863300,0.003422,22
1,Alaska,2,741894,0.001591,0
2,Arizona,4,6931071,0.004774,0
3,Arkansas,5,2988248,0.002440,0
4,California,6,39250017,0.007126,0


In [45]:
#https://covidtracking.com/race/about#download-the-data
# The data were loaded to Githhub so I can read them directly
covid_data=pd.read_csv('https://raw.githubusercontent.com/yaolihua081/COVID-19-Project/main/CRDT%20Data%20-%20CRDTDec5.csv')#data by dec 2
covid_data.dropna()
#this is to match state name
USAStateABV_data=pd.read_csv('https://raw.githubusercontent.com/yaolihua081/Stanford-CS06dataviz/main/USAStateABV.csv')
#USAStateABV_data=pd.read_csv('USAStateABV.csv')
#covid_data=pd.read_csv('Race Data Entry - CRDT.csv')

#find week for the date
covid_data['date']=covid_data['Date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d').strftime('%m/%d/%Y'))
covid_data.head
covid_data['week']=pd.DatetimeIndex(covid_data['date']).week
covid_data
#match state name
USAStateABV_data['state_code']=USAStateABV_data['Code']
covid_data['state_code']=covid_data['State']
covid_data.drop(['State'],axis = 1, inplace=True)

covid_data_sorted=covid_data.sort_values(by='state_code')
USAStateABV_data_sorted=USAStateABV_data.sort_values(by='state_code')
covid_data_sorted=pd.merge(covid_data_sorted,USAStateABV_data_sorted)
covid_data_sorted.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  del sys.path[0]


,Date,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown,Hosp_Total,Hosp_White,Hosp_Black,Hosp_LatinX,Hosp_Asian,Hosp_AIAN,Hosp_NHPI,Hosp_Multiracial,Hosp_Other,Hosp_Unknown,Hosp_Ethnicity_Hispanic,Hosp_Ethnicity_NonHispanic,Hosp_Ethnicity_Unknown,Tests_Total,Tests_White,Tests_Black,Tests_LatinX,Tests_Asian,Tests_AIAN,Tests_NHPI,Tests_Multiracial,Tests_Other,Tests_Unknown,Tests_Ethnicity_Hispanic,Tests_Ethnicity_NonHispanic,Tests_Ethnicity_Unknown,date,week,state_code,State,Abbrev,Code
0,20201202,33797,10472,969.0,NaN,1246.0,6939.0,1037.0,2194.0,3598.0,7342.0,1799.0,16863.0,15135,122.0,41.0,5.0,NaN,9.0,44.0,9.0,3.0,2.0,9.0,1.0,95.0,26.0,768.0,228.0,31.0,NaN,51.0,208.0,87.0,51.0,32.0,80.0,24.0,541.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/02/2020,49,AK,Alaska,Alaska,AK
1,20200617,696,398,17.0,NaN,59.0,78.0,17.0,24.0,17.0,86.0,49.0,540.0,107,12.0,7.0,0.0,NaN,2.0,2.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,55.0,31.0,0.0,NaN,7.0,7.0,5.0,0.0,4.0,1.0,5.0,47.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/17/2020,25,AK,Alaska,Alaska,AK
2,20201018,12016,4072,493.0,NaN,548.0,2519.0,526.0,522.0,452.0,2884.0,964.0,6654.0,4398,67.0,25.0,3.0,NaN,7.0,24.0,7.0,0.0,0.0,1.0,1.0,61.0,5.0,368.0,91.0,15.0,NaN,26.0,116.0,55.0,14.0,12.0,39.0,19.0,275.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/18/2020,42,AK,Alaska,Alaska,AK
3,20200614,661,378,17.0,NaN,58.0,71.0,17.0,24.0,15.0,81.0,48.0,514.0,99,12.0,7.0,0.0,NaN,2.0,2.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/14/2020,24,AK,Alaska,Alaska,AK
4,20201021,12644,4280,516.0,NaN,586.0,2673.0,552.0,573.0,474.0,2990.0,986.0,7034.0,4624,68.0,25.0,3.0,NaN,7.0,24.0,7.0,0.0,0.0,2.0,1.0,61.0,6.0,382.0,95.0,16.0,NaN,28.0,121.0,55.0,15.0,12.0,40.0,19.0,287.0,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/21/2020,43,AK,Alaska,Alaska,AK


In [46]:
#The last date numbers--week 49, 20201202
last_date=covid_data_sorted.sort_values(by='Date', ascending=False)['Date'][0]

covid_data_sorted['state']=covid_data_sorted['State']
covid_data_sorted=covid_data_sorted.sort_values(by='state')
filter=covid_data_sorted['Date']==last_date
covid_data_47=covid_data_sorted[filter]
covid_data_47_sorted=covid_data_47.sort_values(by=['Cases_Total','Deaths_Total'], ascending=False)


covid_data_47_sorted=pd.merge(covid_data_47_sorted,source_sorted)
covid_data_47_sorted.head()


,Date,Cases_Total,Cases_White,Cases_Black,Cases_LatinX,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Other,Cases_Unknown,Cases_Ethnicity_Hispanic,Cases_Ethnicity_NonHispanic,Cases_Ethnicity_Unknown,Deaths_Total,Deaths_White,Deaths_Black,Deaths_LatinX,Deaths_Asian,Deaths_AIAN,Deaths_NHPI,Deaths_Multiracial,Deaths_Other,Deaths_Unknown,Deaths_Ethnicity_Hispanic,Deaths_Ethnicity_NonHispanic,Deaths_Ethnicity_Unknown,Hosp_Total,Hosp_White,Hosp_Black,Hosp_LatinX,Hosp_Asian,Hosp_AIAN,Hosp_NHPI,Hosp_Multiracial,Hosp_Other,Hosp_Unknown,Hosp_Ethnicity_Hispanic,Hosp_Ethnicity_NonHispanic,Hosp_Ethnicity_Unknown,Tests_Total,Tests_White,Tests_Black,Tests_LatinX,Tests_Asian,Tests_AIAN,Tests_NHPI,Tests_Multiracial,Tests_Other,Tests_Unknown,Tests_Ethnicity_Hispanic,Tests_Ethnicity_NonHispanic,Tests_Ethnicity_Unknown,date,week,state_code,State,Abbrev,Code,state,id,population,engineers,hurricanes
0,20201202,82203,55425,1890.0,2904.0,1110.0,9996.0,NaN,NaN,1855.0,9023.0,2904.0,70276.0,9023,995.0,766.0,5.0,10.0,11.0,126.0,NaN,NaN,11.0,66.0,10.0,919.0,66.0,4626.0,3073.0,55.0,120.0,98.0,921.0,NaN,NaN,64.0,295.0,120.0,4211.0,295.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/02/2020,49,SD,South Dakota,S.D.,SD,South Dakota,46,865454,0.003085,0
1,20201202,80135,42900,2432.0,NaN,1131.0,3252.0,NaN,951.0,177.0,29292.0,NaN,NaN,80135,972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,972.0,NaN,NaN,972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/02/2020,49,ND,North Dakota,N.D.,ND,North Dakota,38,757952,0.003404,0
2,20201202,78160,35592,2156.0,NaN,2415.0,1569.0,898.0,1405.0,21305.0,12820.0,24572.0,38069.0,15519,954.0,634.0,25.0,NaN,24.0,18.0,10.0,26.0,102.0,115.0,125.0,634.0,195.0,4745.0,2769.0,166.0,NaN,167.0,128.0,103.0,81.0,977.0,354.0,1116.0,2911.0,718.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/02/2020,49,OR,Oregon,Ore.,OR,Oregon,41,4093465,0.005457,0
3,20201202,748603,252503,70513.0,143795.0,14746.0,864.0,744.0,NaN,25812.0,239626.0,143795.0,365182.0,239626,13397.0,6911.0,2628.0,2177.0,461.0,17.0,12.0,NaN,77.0,1114.0,2177.0,10106.0,1114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10699586.0,3434976.0,799294.0,768787.0,398570.0,8329.0,10950.0,NaN,330013.0,4948667.0,768787.0,4982132.0,4948667.0,12/02/2020,49,IL,Illinois,Ill.,IL,Illinois,17,12801539,0.004478,0
4,20201202,664238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,664238.0,NaN,NaN,664238,34942.0,10735.0,7986.0,8249.0,2170.0,NaN,NaN,NaN,2287.0,3515.0,8249.0,23178.0,3515.0,89995.0,13165.0,16234.0,18170.0,4210.0,NaN,NaN,NaN,NaN,38216.0,18170.0,33609.0,38216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/02/2020,49,NY,New York,N.Y.,NY,New York,36,19745289,0.004941,12


In [47]:
#name_index=covid_data_sorted.columns
name_index=['Date', 'Cases_Total', 'Cases_White', 'Cases_Black', 'Cases_LatinX',
       'Cases_Asian', 'Cases_AIAN', 
        'Cases_Ethnicity_Hispanic',
       'Cases_Ethnicity_NonHispanic',
       'Deaths_Total', 'Deaths_White', 'Deaths_Black', 'Deaths_LatinX',
       'Deaths_Asian', 
        'Deaths_Ethnicity_Hispanic',
       'Deaths_Ethnicity_NonHispanic', 'date', 'week', 'state_code', 'State','state']
name_index1=['Date','State','Cases_Total','Deaths_Total','week']       

In [48]:
# I want to find weekly cases,need to do substraction between rows, but there are NAN numbers, 
#for each state, define weekly cases and death
name_index1=['Date','State','Cases_Total','Deaths_Total','week']  
def computeStateData(covid_data_sorted, state,name_index1):
  covid_data_subset=covid_data_sorted[name_index1]
  covid_data_subset.dropna(axis = 0, how ='any', inplace = True) 
  covid_data_subset=pd.DataFrame(covid_data_subset)
  filter=covid_data_subset['State']==state
  covid_data_subset_state= covid_data_subset[filter]
  covid_data_subset_state=covid_data_subset_state.sort_values(by='Date')
  n=covid_data_subset_state.shape[0]
  case_ls=[int(covid_data_subset_state['Cases_Total'].values[0])]
  death_ls=[int(covid_data_subset_state['Deaths_Total'].values[0])]
  for i in range(1,n):
    case_ls.append(int(int(covid_data_subset_state['Cases_Total'].values[i])-int(covid_data_subset_state['Cases_Total'].values[i-1])))
    death_ls.append(int(int(covid_data_subset_state['Deaths_Total'].values[i])-int(covid_data_subset_state['Deaths_Total'].values[i-1])))

  temp=pd.DataFrame(covid_data_subset_state['week'].values)
  temp1=pd.concat([temp,pd.DataFrame(case_ls)],axis=1)
  state_data=pd.concat([temp1,pd.DataFrame(death_ls)],axis=1)
  state_data.columns=['week','cases','deaths']
  state_data_sum=state_data.groupby('week')['cases'].sum()
  temp=pd.DataFrame(state_data_sum.index.tolist())
  state_data_week_case=pd.concat([temp,pd.DataFrame(state_data_sum.values)],axis=1).reindex(temp.index)
  state_data_week_case.columns=['week','week_cases']
  state_data_week_case
  state_data_sum=state_data.groupby('week')['deaths'].sum()
  temp=pd.DataFrame(state_data_sum.index.tolist())
  state_data_week_death=pd.concat([temp,pd.DataFrame(state_data_sum.values)],axis=1).reindex(temp.index)
  state_data_week_death.columns=['week','week_deaths']
  state_data_week_death.drop(['week'],axis = 1, inplace=True)
  state_data_week=pd.concat([state_data_week_case,state_data_week_death],axis=1)
  n=state_data_week.shape[0]
  name_state=[]
  
  for i in range(n):
    name_state.append(state)
  all=pd.concat([pd.DataFrame(name_state),pd.DataFrame(state_data_week)], axis=1)
  all.columns=['state','week','cases','deaths']
  
  return all


In [49]:
name_index1=['Date','State','Cases_Total','Deaths_Total','week']  

dat=computeStateData(covid_data_sorted,'California',  name_index1)
all_state_week=dat
states=['Texas','Florida','Ohio','Georgia','Iowa','New York', 'New Jersey','Pennsylvania','Illinois','Washington','Wisconsin','Colorado','Michigan']
for state in states:
      dat=computeStateData(covid_data_sorted,state,  name_index1)
      all_state_week=pd.concat([all_state_week, dat],axis=0)

all_state_week  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying

,state,week,cases,deaths
0,California,15,21794,651
1,California,16,7169,421
2,California,17,13201,575
3,California,18,11452,491
4,California,19,13064,519
...,...,...,...,...
30,Michigan,45,31597,246
31,Michigan,46,46789,431
32,Michigan,47,53229,499
33,Michigan,48,49131,592


In [50]:
brush=alt.selection(type='interval')
yrange = [0, 100000]
state='California'
filter=all_state_week['state']==state

bar1 = alt.Chart(all_state_week[filter], title="California Covid-19 Cases by Week").mark_bar().encode(
     x='week:N',
     y=alt.Y('cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='deaths:Q'
     #color=alt.condition(brush, 'age_group:N', alt.value('colors'))
).add_selection(brush)
state='Texas'
filter=all_state_week['state']==state
bar2 = alt.Chart(all_state_week[filter],title="Texas Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('cases:Q',scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='deaths:Q'
).transform_filter(brush)
state='Florida'
filter=all_state_week['state']==state
bar3 = alt.Chart(all_state_week[filter],title="Florida Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('cases:Q',scale=alt.Scale(domain=(0, 100000)), title="Total Cases"),
     color='deaths:Q'
).transform_filter(brush)

state='Georgia'
filter=all_state_week['state']==state
bar4 = alt.Chart(all_state_week[filter],title="Georgia Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='deaths:Q'
).transform_filter(brush)
state='New York'
filter=all_state_week['state']==state
bar5 = alt.Chart(all_state_week[filter],title="New York Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('cases:Q', title="Total Cases"),
     color='deaths:Q'
).transform_filter(brush)

state='Ohio'
filter=all_state_week['state']==state
bar6 = alt.Chart(all_state_week[filter],title="Ohio Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='deaths:Q'
).transform_filter(brush)

state='Michigan'
filter=all_state_week['state']==state
bar7 = alt.Chart(all_state_week[filter],title="Michigan Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='deaths:Q'
).transform_filter(brush)

state='Colorado'
filter=all_state_week['state']==state
bar8 = alt.Chart(all_state_week[filter],title="Colorado Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='deaths:Q'
).transform_filter(brush)

combined=bar5 & bar1 &bar2 &bar3| bar4   & bar6 & bar7 &bar8

combined.save('State_Covid_week.html')
from google.colab import files
files.download('State_Covid_week.html')
combined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

In [36]:
# I want to find weekly cases,need to do substraction between rows, but there are NAN numbers, 
#for each state, define weekly cases and death
#Check race and Ethnic

name_index1=['Date','State','Cases_Ethnicity_Hispanic','Deaths_Ethnicity_Hispanic','Cases_Black','Deaths_Black','Cases_White','Deaths_White','Cases_Asian','Deaths_Asian','Cases_Total','Deaths_Total','week']  
def computedat(covid_data_sorted, state, race, index1):
  covid_data_subset=covid_data_sorted[name_index1]
  #covid_data_subset=covid_data_subset.dropna()
  covid_data_subset.dropna(axis = 0, how ='any', inplace = True) 
  covid_data_subset=pd.DataFrame(covid_data_subset)
  filter=covid_data_subset['State']==state
  covid_data_subset_state= covid_data_subset[filter]
  covid_data_subset_state=covid_data_subset_state.sort_values(by='Date')
  n=covid_data_subset_state.shape[0]
  if n>0:
    case_ls=[int(covid_data_subset_state['Cases_'+race].values[0])]
    death_ls=[int(covid_data_subset_state['Deaths_'+race].values[0])]
    if n >=1:
      for i in range(1,n):
        case_ls.append(int(int(covid_data_subset_state['Cases_'+race].values[i])-int(covid_data_subset_state['Cases_'+race].values[i-1])))
        death_ls.append(int(int(covid_data_subset_state['Deaths_'+race].values[i])-int(covid_data_subset_state['Deaths_'+race].values[i-1])))
    temp=pd.DataFrame(covid_data_subset_state['week'].values)
    state_data=pd.concat([temp,pd.DataFrame(case_ls),pd.DataFrame(death_ls)],axis=1)
    state_data.columns=['week','Cases','Deaths']
    state_data_sum=state_data.groupby('week')['Cases'].sum()
    temp=pd.DataFrame(state_data_sum.index.tolist())
    state_data_week_case=pd.concat([temp,pd.DataFrame(state_data_sum.values)],axis=1).reindex(temp.index)
    state_data_week_case.columns=['week','week_cases']
    state_data_sum=state_data.groupby('week')['Deaths'].sum()
    temp=pd.DataFrame(state_data_sum.index.tolist())
    state_data_week_death=pd.concat([temp,pd.DataFrame(state_data_sum.values)],axis=1).reindex(temp.index)
    state_data_week_death.columns=['week', 'week_deaths']
    state_data_week_death.drop(['week'],axis = 1, inplace=True)
    state_data_week=pd.concat([state_data_week_case,state_data_week_death],axis=1)
    temp_state=state_data_week
    n=temp_state.shape[0]
    name_state=[]
    Ethnic=[]
    for i in range(n):
      name_state.append(state)
      Ethnic.append(race) 
    all=pd.concat([pd.DataFrame(name_state),pd.DataFrame(Ethnic)], axis=1)
    all.columns=['State','Ethnic']
    all_state=pd.concat([all,temp_state],axis=1)
    return all_state
  return   



In [51]:
name_index1=['Date','State','Cases_Ethnicity_Hispanic','Deaths_Ethnicity_Hispanic','Cases_Black','Deaths_Black','Cases_White','Deaths_White','week']  
index1=['Date','State','Cases_Ethnicity_Hispanic','Deaths_Ethnicity_Hispanic','week']  
dat=computedat(covid_data_sorted,'California', 'Ethnicity_Hispanic', index1)
all_state=dat
races=['Black','White']
states=['California']
for state in states:
  for race in races:
    index1=['Date','State','Cases_'+race,'Deaths_'+race,'week']
    dat=computedat(covid_data_sorted,state, race, name_index1)
    all_state=pd.concat([all_state, dat],axis=0)


races=['Black','White','Ethnicity_Hispanic']
states=['Texas','Florida','Ohio','Georgia','Iowa','New York', 'New Jersey','Pennsylvania','Illinois','Washington','Wisconsin','Colorado','Michigan']
#states=['Texas','Florida']
for state in states:
  for race in races:
      index1=['Date','State','Cases_'+race,'Deaths_'+race,'week'] 
      dat=computedat(covid_data_sorted,state, race,index1)
      all_state=pd.concat([all_state, dat],axis=0)


all_state
#'Ohio','Georgia','Iowa','New York', 'New Jersey','Pennsylvania','Illinois','Washington','Wisconsin','Colorado','Michigan'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dis

,State,Ethnic,week,week_cases,week_deaths
0,California,Ethnicity_Hispanic,16,7567,298
1,California,Ethnicity_Hispanic,17,4525,204
2,California,Ethnicity_Hispanic,18,5035,202
3,California,Ethnicity_Hispanic,19,5398,210
4,California,Ethnicity_Hispanic,20,6683,223
...,...,...,...,...,...
30,Michigan,Ethnicity_Hispanic,45,1458,8
31,Michigan,Ethnicity_Hispanic,46,1908,21
32,Michigan,Ethnicity_Hispanic,47,2021,12
33,Michigan,Ethnicity_Hispanic,48,1929,20


In [52]:
all_state_sorted=all_state.sort_values(by='State')
all_state_sorted
all_state_sorted['week_case']=all_state_sorted['week_cases'].apply(lambda x: x if x > 0 else 0)
all_state_sorted['week_death']=all_state_sorted['week_deaths'].apply(lambda x: x if x > 0 else 0)

In [53]:
brush=alt.selection(type='interval')
filter=all_state_sorted['Ethnic']=='Ethnicity_Hispanic'
all_state_sub=all_state_sorted[filter]
bar1 = alt.Chart(all_state_sub, title="USA Covid-19 Cases Ethnicity_Hispanic by Dec 2").mark_bar().encode(
     x='week:N',
     y=alt.Y('week_case:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='State:N',
     tooltip=['State:N',
              alt.Tooltip('week_case:Q'),
              alt.Tooltip('week_death:Q')
     ]
).add_selection(brush)

bar2 = alt.Chart(all_state_sub, title="USA Covid-19 deaths Ethnicity_Hispanic by Dec 2").mark_bar().encode(
    x='week:N',
    y=alt.Y('week_death:Q', scale=alt.Scale(domain=(0, 4500)),title="Total Deaths"),
    color='State:N',
    tooltip=['State:N',
             alt.Tooltip('week_case:Q'),
             alt.Tooltip('week_death:Q')
             ]
#).transform_filter(brush)
).add_selection(brush)

combined1=bar1 | bar2



combined1.save('USA_Covid_State_Hispanic_weekly.html')
from google.colab import files
files.download('USA_Covid_State_Hispanic_weekly.html')
combined1

#sns.relplot(x='week',y='week_cases',col='State', hue='Ethnic', data=all_state )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

In [31]:
brush=alt.selection(type='interval')
filter=all_state_sorted['Ethnic']=='Black'
all_state_sub=all_state_sorted[filter]
bar1 = alt.Chart(all_state_sub, title="USA Covid-19 Cases Black by Dec 2").mark_bar().encode(
     x='week:N',
     y=alt.Y('week_case:Q', title="Total Cases"),
     color='State:N',
     tooltip=['State:N',
              alt.Tooltip('week_case:Q'),
              alt.Tooltip('week_deaths:Q')
     ]
).add_selection(brush)

bar2 = alt.Chart(all_state_sub,title="USA Covid-19 deaths Black by Dec 2").mark_bar().encode(
    x='week:N',
    y=alt.Y('week_death:Q', title="Total Deaths"),
    color='State:N',
    tooltip=['State:N',
             alt.Tooltip('week_case:Q'),
             alt.Tooltip('week_death:Q')
             ]
#).transform_filter(brush)
).add_selection(brush)

combined1=bar1 | bar2 



combined1.save('USA_Covid_State_Black_weekly.html')
from google.colab import files
files.download('USA_Covid_State_Black_weekly.html')
combined1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

In [40]:
brush=alt.selection(type='interval')
filter=all_state_sorted['Ethnic']=='White'
all_state_sub=all_state_sorted[filter]
bar1 = alt.Chart(all_state_sub, title="USA Covid-19 Cases White by Dec 2").mark_bar().encode(
     x='week:N',
     y=alt.Y('week_case:Q', title="Total Cases"),
     color='State:N',
     tooltip=['State:N',
              alt.Tooltip('week_case:Q'),
              alt.Tooltip('week_deaths:Q')
     ]
).add_selection(brush)

bar2 = alt.Chart(all_state_sub,title="USA Covid-19 deaths White by Dec 2").mark_bar().encode(
    x='week:N',
    y=alt.Y('week_death:Q', title="Total Deaths"),
    color='State:N',
    tooltip=['State:N',
             alt.Tooltip('week_case:Q'),
             alt.Tooltip('week_death:Q')
             ]
#).transform_filter(brush)
).add_selection(brush)

combined1=bar1 | bar2 



combined1.save('USA_Covid_State_White_weekly.html')
from google.colab import files
files.download('USA_Covid_State_White_weekly.html')
combined1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

In [41]:
brush=alt.selection(type='interval')



bar3 = alt.Chart(all_state_sorted,title="USA Covid-19 Cases by Ethnic by Dec 2").mark_bar().encode(
    x='week:N',
    y=alt.Y('week_case:Q', title="Total Cases"),
    color='Ethnic:N',
    tooltip=['State:N',
             alt.Tooltip('week_case:Q'),
             alt.Tooltip('week_deaths:Q')]
).add_selection(brush)

bar4 = alt.Chart(all_state_sorted,title="USA Covid-19 deaths by Ethnic by Dec 2").mark_bar().encode(
    x='week:N',
    y=alt.Y('week_deaths:Q', title="Total Deaths"),
    color='Ethnic:N',
    tooltip=['State:N',
             alt.Tooltip('week_case:Q'),
             alt.Tooltip('week_deaths:Q')]
).add_selection(brush)

combined =bar3 |bar4
combined
combined.save('USA_Covid_Race_Ethnic_weekly.html')
from google.colab import files
files.download('USA_Covid_Race_Ethnic_weekly.html')
combined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

In [55]:
covid_data_47_sorted['Cases_Ratio']=round(100*covid_data_47_sorted['Cases_Total'].apply(lambda x : int(x))/covid_data_47_sorted['population'].apply(lambda x : int(x)),2)
covid_data_47_sorted['Deaths_Ratio']=round(100*covid_data_47_sorted['Deaths_Total'].apply(lambda x : int(x))/covid_data_47_sorted['Cases_Total'].apply(lambda x : int(x)),2)#covid_data_47_sorted['Black_Deaths_Ratio']=100*covid_data_47_sorted['Deaths_Black'].apply(lambda x : x if np.nan else int(x))/covid_data_47_sorted['Cases_Black'].apply(lambda x : int(x)))
covid_data_47_sorted['Hispanic_Deaths_Ratio']=round(100*covid_data_47_sorted['Deaths_Ethnicity_Hispanic'].apply(lambda x : x if np.nan else int(x))/covid_data_47_sorted['Cases_Ethnicity_Hispanic'].apply(lambda x : x if np.nan else int(x)),2)
#covid_data_47_sorted['White_Deaths_Ratio']=round(100*covid_data_47_sorted['Deaths_White'].apply(lambda x : x if np.nan else int(x))/covid_data_47_sorted['Cases_White'].apply(lambda x : x if np.nan else int(x)),2)
covid_data_47_sorted['Black_Deaths_Ratio']=round(100*covid_data_47_sorted['Deaths_Black'].apply(lambda x : x if np.nan else int(x))/covid_data_47_sorted['Cases_Black'].apply(lambda x : x if np.nan else int(x)),2)
covid_data_47_sorted['Asian_Deaths_Ratio']=round(100*covid_data_47_sorted['Deaths_Asian'].apply(lambda x : x if np.nan else int(x))/covid_data_47_sorted['Cases_Asian'].apply(lambda x : x if np.nan else int(x)),2)
covid_data_47_sorted['Ethnicity_NonHispanic_Deaths_Ratio']=round(100*covid_data_47_sorted['Deaths_Ethnicity_NonHispanic'].apply(lambda x : x if np.nan else int(x))/covid_data_47_sorted['Cases_Ethnicity_NonHispanic'].apply(lambda x : x if np.nan else int(x)),2)



In [56]:

covid_data_47_sorted_Hispanic=covid_data_47_sorted.sort_values(by='Hispanic_Deaths_Ratio')
covid_data_47_sorted_Hispanic_sub=covid_data_47_sorted_Hispanic[-20:]
covid_data_47_sorted_Hispanic_sub

covid_data_47_sorted_Black=covid_data_47_sorted.sort_values(by='Black_Deaths_Ratio')
covid_data_47_sorted_Black_sub=covid_data_47_sorted_Black[-20:]


In [40]:
brush=alt.selection(type='interval')
bar1 = alt.Chart(covid_data_47_sorted_Hispanic_sub,title="Hispanic Death Rate for Top 20 State by Dec 2").mark_bar().encode(
    x='State:N',
    y=alt.Y('Hispanic_Deaths_Ratio:Q', scale=alt.Scale(domain=(0, 50)),title="Hispanic Death Ratio"),
    color='Deaths_Total:Q',
    tooltip=['State:N',
             alt.Tooltip('Hispanic_Deaths_Ratio:Q'),
             alt.Tooltip('Black_Deaths_Ratio:Q'),
             alt.Tooltip('Deaths_Ratio:Q'),
             alt.Tooltip('Deaths_Total:Q'),
             alt.Tooltip('Cases_Total:Q')]
).add_selection(brush)
bar2 = alt.Chart(covid_data_47_sorted_Black_sub,title="Black Death Rate for Top 20 State by Dec 2").mark_bar().encode(
    x='State:N',
    y=alt.Y('Black_Deaths_Ratio:Q',scale=alt.Scale(domain=(0, 50)), title="Black Death Ratio"),
    color='Deaths_Total:Q',
    tooltip=['State:N',
             alt.Tooltip('Hispanic_Deaths_Ratio:Q'),
             alt.Tooltip('Black_Deaths_Ratio:Q'),
             alt.Tooltip('Deaths_Ratio:Q'),
             alt.Tooltip('Deaths_Total:Q'),
             alt.Tooltip('Cases_Total:Q')]
).add_selection(brush)

combined =bar1 |bar2
combined
combined.save('USA_Covid_Black_Hispanic.html')
from google.colab import files
files.download('USA_Covid_Black_Hispanic.html')
combined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

Cases numer rank: Californai, Texas,Florida, Illinois, New York, Georgia
Death number rank :New York, Texas, California, Florida, Illinois,Pennsvia
Case ratio rank: 
Death ratio Rank 
New York: White , 1/3,
Texas: Hispanic count half, then white,
California: Hispanic count half, then white.
Florida: White half, then hispanic
New Jersy: white half
Illinois: white half
Georgia: white, then black(1/3)
Pennsynia: almost all white 7/9, then black
Michigen: white,5/8,  then black


In [57]:
states = alt.topo_feature(data.us_10m.url, 'states')
states
# Stae name and their IDS, with their populations
source_sorted=source.sort_values(by='state')
source_sorted.head()
bar1=alt.Chart(states, title="USA State Death Distributions by Race/Ethnic By Dec 5, 2020").mark_geoshape().encode(
            alt.Color('Deaths_Total', type='quantitative'),
            tooltip=['state:N',
                 alt.Tooltip('Deaths_Total:Q'),
                 alt.Tooltip('Deaths_White:Q'),
                 alt.Tooltip('Deaths_Black:Q'),
                 alt.Tooltip('Deaths_Ethnicity_Hispanic:Q'),
                 alt.Tooltip('Deaths_Ethnicity_NonHispanic:Q'),
                 alt.Tooltip('Deaths_Asian:Q')]
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(covid_data_47_sorted, 'id', ['state','Deaths_Total','Deaths_White','Deaths_Black','Deaths_Asian','Deaths_Ethnicity_Hispanic','Deaths_Ethnicity_NonHispanic'])
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
)
combined=bar1
combined.save('USA_Covid_Geo_Death_Race_Ethnic.html')
from google.colab import files
files.download('USA_Covid_Geo_Death_Race_Ethnic.html')
combined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.Chart(...)

In [62]:
bar1=alt.Chart(states, title="USA State Cases Distributions by Race/Ethnic By Dec 5, 2020").mark_geoshape().encode(
            alt.Color('Cases_Total', type='quantitative'),
            tooltip=['state:N',
                 alt.Tooltip('Cases_Total:Q'),
                 alt.Tooltip('Cases_White:Q'),
                 alt.Tooltip('Cases_Black:Q'),
                 alt.Tooltip('Cases_Ethnicity_Hispanic:Q'),
                 alt.Tooltip('Cases_Ethnicity_NonHispanic:Q'),
                 alt.Tooltip('Cases_Ethnicity_Unknow:Q'),
                 alt.Tooltip('Cases_Asian:Q')]
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(covid_data_47_sorted, 'id', ['state','Cases_Total','Cases_White','Cases_Black','Cases_Asian','Cases_Ethnicity_Hispanic','Cases_Ethnicity_NonHispanic','Cases_Ethnicity_Unknown'])
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
)
combined=bar1
combined.save('USA_Covid_Geo_Case_Race_Ethnic.html')
from google.colab import files
files.download('USA_Covid_Geo_Case_Race_Ethnic.html')
combined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.Chart(...)

In [ ]:
bar1=alt.Chart(states, title="USA State Cases, Deaths and Death Ratios: Distributions by Race/Ethnic By Nov 22, 2020").mark_geoshape().encode(
            alt.Color('Cases_Ratio', type='quantitative'),
            tooltip=['state:N',
                 alt.Tooltip('Cases_Total:Q'),
                 alt.Tooltip('Cases_White:Q'),
                 alt.Tooltip('Cases_Black:Q'),
                 alt.Tooltip('Cases_Ethnicity_Hispanic:Q'),
                 alt.Tooltip('Cases_Ethnicity_NonHispanic:Q'),
                 alt.Tooltip('Cases_Ethnicity_Unknow:Q'),
                 alt.Tooltip('Cases_Asian:Q')
                  ]
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(covid_data_47_sorted, 'id', ['state','Cases_Ratio','Cases_Total','Cases_White','Cases_Black','Cases_Asian','Cases_Ethnicity_Hispanic','Cases_Ethnicity_NonHispanic','Cases_Ethnicity_Unknown'])
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
)
combined=bar1
combined.save('USA_Covid_Geo_Case_Ratio_Race_Ethnic.html')
from google.colab import files
files.download('USA_Covid_Geo_Case_Ratio_Race_Ethnic.html')
combined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.Chart(...)

In [ ]:
bar1=alt.Chart(states, title="USA State Death Ratios: Distributions by Race/Ethnic By Nov 22, 2020").mark_geoshape().encode(
            alt.Color('Deaths_Ratio', type='quantitative'),
            tooltip=['state:N',
                 alt.Tooltip('Cases_Ratio:Q'),
                 alt.Tooltip('Deaths_Ratio:Q'),
                 alt.Tooltip('Black_Deaths_Ratio:Q'),
                 alt.Tooltip('Hispanic_Deaths_Ratio:Q'),
                 alt.Tooltip('Ethnicity_NonHispanic_Deaths_Ratio:Q'),
                 alt.Tooltip('Asian_Deaths_Ratio:Q')
                  ]
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(covid_data_47_sorted, 'id', ['state','Deaths_Ratio','Cases_Total','Cases_Ratio','Deaths_Ratio','Asian_Deaths_Ratio','Hispanic_Deaths_Ratio','Black_Deaths_Ratio','Ethnicity_NonHispanic_Deaths_Ratio'])
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
)
combined=bar1
combined.save('USA_Covid_Geo_Death_Ratio_Race_Ethnic.html')
from google.colab import files
files.download('USA_Covid_Geo_Death_Ratio_Race_Ethnic.html')
combined

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.Chart(...)

In [59]:
United_States_Covid=pd.read_csv('https://raw.githubusercontent.com/yaolihua081/Stanford-CS06dataviz/main/United_States_COVIDDec5.csv')
United_States_Covid.head()

United_States_Covid.dropna()
United_States_Covid.head()
United_state_sum=United_States_Covid.groupby(['state'])['new_case'].sum()

temp=pd.DataFrame(United_state_sum)
temp=pd.DataFrame(temp.index.tolist())

United_state_covid_case_data=pd.concat([temp,pd.DataFrame(United_state_sum.values)],axis=1).reindex(temp.index)

United_state_covid_case_data.columns=['State','total_cases']
United_state_covid_case_data.head()

United_state_sum=United_States_Covid.groupby(['state'])['new_death'].sum()

temp=pd.DataFrame(United_state_sum)
temp=pd.DataFrame(temp.index.tolist())

United_state_covid_death_data=pd.concat([temp,pd.DataFrame(United_state_sum.values)],axis=1).reindex(temp.index)

United_state_covid_death_data.columns=['State','total_deaths']

United_state_covid_case_data=United_state_covid_case_data.sort_values(by='State')

United_state_covid_death_data=United_state_covid_death_data.sort_values(by='State')
United_state_covid_death_data.drop(['State'],axis = 1, inplace=True)
United_State_covid_data=pd.concat([United_state_covid_case_data,United_state_covid_death_data],axis=1)
United_State_covid_data


United_States_Covid['week']=pd.DatetimeIndex(United_States_Covid['submission_date']).week
United_state_sum=United_States_Covid.groupby(['state','week'])['new_case'].sum()


temp=pd.DataFrame(United_state_sum)
temp=pd.DataFrame(temp.index.tolist())


United_state_covid_case_week_data=pd.concat([temp,pd.DataFrame(United_state_sum.values)],axis=1).reindex(temp.index)

United_state_covid_case_week_data.columns=['state','week','total_cases']
United_state_covid_case_week_data.head()

United_state_sum=United_States_Covid.groupby(['state','week'])['new_death'].sum()

temp=pd.DataFrame(United_state_sum)
temp=pd.DataFrame(temp.index.tolist())

United_state_covid_death_week_data=pd.concat([temp,pd.DataFrame(United_state_sum.values)],axis=1).reindex(temp.index)

United_state_covid_death_week_data.columns=['state','week','total_deaths']

United_state_covid_case_dweek_ata=United_state_covid_case_week_data.sort_values(by='state')

United_state_covid_death_week_data=United_state_covid_death_week_data.sort_values(by='state')
United_state_covid_death_week_data.drop(['state','week'],axis = 1, inplace=True)
United_State_covid_week_data=pd.concat([United_state_covid_case_week_data,United_state_covid_death_week_data],axis=1)
United_State_covid_week_data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)


,state,week,total_cases,total_deaths
0,AK,4,0,0
1,AK,5,0,0
2,AK,6,0,0
3,AK,7,0,0
4,AK,8,0,0
...,...,...,...,...
2755,WY,45,3587,27
2756,WY,46,5184,30
2757,WY,47,5675,32
2758,WY,48,4320,39


In [60]:
filter=United_State_covid_data['total_cases']>=200000
United_State_covid_data_sub=United_State_covid_data[filter]
state_sub=United_State_covid_data_sub['State'].values

United_State_covid_week_data['state'].values
filter=United_State_covid_week_data['state'].apply(lambda x : True if x in state_sub else False)

United_State_covid_week_data_sub=United_State_covid_week_data[filter]
#United_State_covid_week_data_sub=United_State_covid_week_data_sub.sort_values(by='week')
#United_State_covid_week_data_sub
brush=alt.selection(type='interval')
bar1 = alt.Chart(United_State_covid_week_data_sub, title=" Covid-19 Cases for each week for State with minimum of 200000 Cases by Dec 5").mark_bar().encode(
     x=alt.X('week:Q',title="Week"),
     y=alt.Y('total_cases:Q', title="Total Casess"),
     color='state:N',
     tooltip=['state:N',
              alt.Tooltip('total_cases:Q'),
              alt.Tooltip('total_deaths:Q')]
).add_selection(brush)

bar2 = alt.Chart(United_State_covid_data_sub, title="USA Covid-19 Cases for State with minimum of 200000 Cases by Dec 5").mark_bar().encode(
     x='State:N',
     y=alt.Y('total_cases:Q', title="Total Cases"),
     color='total_deaths:Q',
     tooltip=['State:N',
              alt.Tooltip('total_cases:Q'),
              alt.Tooltip('total_deaths:Q')]
#).add_selection(brush)
).transform_filter(brush)
combined1=bar1 | bar2 



combined1.save('USA_Covid_Top_State.html')
from google.colab import files
files.download('USA_Covid_Top_State.html')
combined1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

In [61]:
brush=alt.selection(type='interval')
yrange = [0, 100000]
state='CA'
filter=United_State_covid_week_data['state']==state

bar1 = alt.Chart(United_State_covid_week_data[filter], title="California Covid-19 Cases by Week").mark_bar().encode(
     x='week:N',
     y=alt.Y('total_cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='total_deaths:Q'
     #color=alt.condition(brush, 'age_group:N', alt.value('colors'))
).add_selection(brush)
state='TX'
filter=United_State_covid_week_data['state']==state
bar2 = alt.Chart(United_State_covid_week_data[filter],title="Texas Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('total_cases:Q',scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='total_deaths:Q'
).transform_filter(brush)
state='FL'
filter=United_State_covid_week_data['state']==state
bar3 = alt.Chart(United_State_covid_week_data[filter],title="Florida Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('total_cases:Q',scale=alt.Scale(domain=(0, 100000)), title="Total Cases"),
     color='total_deaths:Q'
).transform_filter(brush)

state='GA'
filter=United_State_covid_week_data['state']==state
bar4 = alt.Chart(United_State_covid_week_data[filter],title="Georgia Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('total_cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='total_deaths:Q'
).transform_filter(brush)
state='NY'
filter=United_State_covid_week_data['state']==state
bar5 = alt.Chart(United_State_covid_week_data[filter],title="New York Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('total_cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='total_deaths:Q'
).transform_filter(brush)

state='OH'
state='IL'
filter=United_State_covid_week_data['state']==state
bar6 = alt.Chart(United_State_covid_week_data[filter],title="IL Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('total_cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='total_deaths:Q'
).transform_filter(brush)

state='WI'
filter=United_State_covid_week_data['state']==state
bar7 = alt.Chart(United_State_covid_week_data[filter],title="WI Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('total_cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='total_deaths:Q'
).transform_filter(brush)

state='CO'
state='NJ'
filter=United_State_covid_week_data['state']==state
bar8 = alt.Chart(United_State_covid_week_data[filter],title="New Jersy Covid-19 Cases by Week").mark_bar().encode(
    x='week:N',
    y=alt.Y('total_cases:Q', scale=alt.Scale(domain=(0, 100000)),title="Total Cases"),
     color='total_deaths:Q'
).transform_filter(brush)

combined=bar5 & bar1 &bar2 &bar3| bar4   & bar6 & bar7 &bar8

combined.save('UAS_State_Covid_week.html')
from google.colab import files
files.download('UAS_State_Covid_week.html')
combined


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

alt.HConcatChart(...)

In [8]:
[United_State_covid_data['total_cases'].sum(),United_State_covid_data['total_deaths'].sum()]

[14255535, 277825]

In [14]:
#United_State_covid_data
filter=United_States_Covid['state']=='CA'
United_States_Covid[filter]

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,week
10494,01/22/2020,CA,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Not agree,Not agree,4
10495,01/23/2020,CA,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Not agree,Not agree,4
10496,01/24/2020,CA,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Not agree,Not agree,4
10497,01/25/2020,CA,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Not agree,Not agree,4
10498,01/26/2020,CA,2,NaN,NaN,2,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Not agree,Not agree,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10807,11/30/2020,CA,1212968,NaN,NaN,14034,0.0,19141,NaN,NaN,20,0.0,12/01/2020 03:26:07 PM,Not agree,Not agree,49
10808,12/01/2020,CA,1225189,NaN,NaN,12221,0.0,19211,NaN,NaN,70,0.0,12/02/2020 03:48:48 PM,Not agree,Not agree,49
10809,12/02/2020,CA,1245948,NaN,NaN,20759,0.0,19324,NaN,NaN,113,0.0,12/03/2020 03:21:08 PM,Not agree,Not agree,49
10810,12/03/2020,CA,1264539,NaN,NaN,18591,0.0,19437,NaN,NaN,113,0.0,12/04/2020 03:00:59 PM,Not agree,Not agree,49


In [37]:
filter=United_State_covid_week_data_sub['state']=="WI"
United_State_covid_week_data_sub[filter]

,state,week,total_cases,week,total_deaths
2622,WI,4,0,4,0
2623,WI,5,0,5,0
2624,WI,6,0,6,0
2625,WI,7,0,7,0
2626,WI,8,0,8,0
2627,WI,9,0,9,0
2628,WI,10,1,10,0
2629,WI,11,26,11,0
2630,WI,12,354,12,4
2631,WI,13,608,13,9


In [11]:
filter=United_State_cocid['state']=="CA"
United_State_covid_week_data_sub[filter]

,state,week,total_cases,week,total_deaths
230,CA,4,2,4,0
231,CA,5,1,5,0
232,CA,6,3,6,0
233,CA,7,2,7,0
234,CA,8,2,8,0
235,CA,9,2,9,0
236,CA,10,98,10,1
237,CA,11,259,11,4
238,CA,12,1340,12,27
239,CA,13,4030,13,105
